# This notebook contains code for training and feature vectors calculating using recurrent autoencoder

In [1]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, dataloader
from statistics import mean
import visdom

In [33]:
class PredictionDataset(Dataset):
    def __init__(self, dataset_type='train'):
        super().__init__()
        if dataset_type=='train':
            self.filenames = [s.split('.')[0] for s in pd.read_csv('./metrics.csv')['Case'].tolist()]
        elif dataset_type=='test':
            self.filenames = [s.split('.')[0] for s in pd.read_csv('../Dataset/SecretPart_dummy.csv')['Case'].tolist()]
        else:
            raise Exception('Unk dataset type!')
        self.pred_info = (pd.read_csv('../Dataset/DX_TEST_RESULT_FULL.csv')
                          .sort_values(by=['file_name', ' user_name', ' xcenter', ' ycenter']))
    
    def __len__(self):
        return len(self.filenames) * 4
    
    def __getitem__(self, idx):
        filename = self.filenames[idx % len(self.filenames)]
        if idx // len(self.filenames) == 0:
            user_name = 'Expert'
        elif idx // len(self.filenames) == 1:
            user_name = 'sample_1'
        elif idx // len(self.filenames) == 2:
            user_name = 'sample_2'
        elif idx // len(self.filenames) == 3:
            user_name = 'sample_3'
            
        info_slice = self.pred_info[self.pred_info['file_name'] == filename][self.pred_info[' user_name'] == user_name]
        
        res_info = []
        for i, row in info_slice.iterrows():
            res_info.append([
                row[' xcenter'] / 1024,
                row[' ycenter'] / 1024,
                row[' rhorizontal'] / 1024,
                row[' rvertical'] / 1024,
            ])
            
        return np.array(res_info, dtype=np.float32)

In [6]:
class GruAutoencoder(nn.Module):
    def __init__(self, d_h):
        super().__init__()
        self.d_h = d_h
        self.d_i = 4
        self.h_0 = nn.Parameter(torch.randn(1, self.d_h), requires_grad=True)
        self.start_token = nn.Parameter(torch.randn(1, self.d_i), requires_grad=True)
        self.encoder = nn.GRUCell(self.d_i, self.d_h)
        self.out_nn = nn.Linear(self.d_h, self.d_i)
        self.decoder = nn.GRUCell(self.d_i, self.d_h)
        
    def encode(self, x):
        r'''
        x.size() == [bs, seq_len, d_i]
        '''
        h_c = self.h_0
        for i in range(x.size(1)):
            h_c = self.encoder(x[:, i, :], h_c)
        
        return h_c / torch.norm(h_c, dim=1, keepdim=True)
    
    def decode(self, h_c, seq_len):
        res = []
        c_token = self.start_token
        for i in range(seq_len):
            h_c = self.decoder(c_token, h_c)
            c_token = self.out_nn(h_c)
            res.append(c_token)
        return torch.stack(res, dim=1)
    
    def forward(self, x):
        seq_len = x.size(1)
        encoded = self.encode(x)
        decoded = self.decode(encoded, seq_len)
        return decoded

# Training

In [9]:
train_dataset = PredictionDataset()
train_loader = dataloader.DataLoader(train_dataset, batch_size=1, shuffle=True)

In [10]:
model = GruAutoencoder(16)
model_opt = torch.optim.Adam(model.parameters())

In [11]:
vis = visdom.Visdom(env='Seq autoencoder train (1)')
step = 0

Setting up a new session...


In [13]:
model.load_state_dict(torch.load('./ae_model.pth'))
model_opt.load_state_dict(torch.load('./ae_model_opt.pth'))

In [10]:
losses = []
for epoch in range(300):
    for batch in train_loader:
        seq_len = batch.size(1)
        if seq_len > 0:
            model_opt.zero_grad()
            prediction = model(batch)
            loss = nn.functional.mse_loss(prediction, batch, reduction='sum')
            loss.backward()
            model_opt.step()

            losses.append(loss.detach().item() / seq_len)

            if step % 20 == 0:
                vis.line(X=[step], Y=[mean(losses)], update='append', name='total loss', win='losses')
                losses = []

            step += 1

c:\users\dmitry\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
torch.save(model.state_dict(), './ae_model.pth')
torch.save(model_opt.state_dict(), './ae_model_opt.pth')

# Inference

In [15]:
rev_idx = {fn: i for i, fn in enumerate(train_dataset.filenames)}

In [17]:
marks = pd.read_csv('../Dataset/OpenPart.csv')
marks.head()

,Case,Sample 1,Sample 2,Sample 3
0,00000072_000.png,1,5,1
1,00000150_002.png,5,5,3
2,00000181_061.png,4,4,3
3,00000211_019.png,4,4,2
4,00000211_041.png,3,5,2


In [22]:
r'''
Calculate feature vectors for predictions in dataset
'''

vectors = dict()

with torch.no_grad():
    for i, row in marks.iterrows():
        fname = row['Case'].split('.')[0]

        pred_exp = train_dataset[rev_idx[fname]]
        pred_1 = train_dataset[rev_idx[fname] + len(train_dataset.filenames)]
        pred_2 = train_dataset[rev_idx[fname] + 2 * len(train_dataset.filenames)]
        pred_3 = train_dataset[rev_idx[fname] + 3 * len(train_dataset.filenames)]
        
        feature_exp = model.encode(torch.tensor(pred_exp).unsqueeze(0)).numpy()
        feature_1 = model.encode(torch.tensor(pred_1).unsqueeze(0)).numpy()
        feature_2 = model.encode(torch.tensor(pred_2).unsqueeze(0)).numpy()
        feature_3 = model.encode(torch.tensor(pred_3).unsqueeze(0)).numpy()
        
        vectors[fname] = [feature_exp, feature_1, feature_2, feature_3]

c:\users\dmitry\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [23]:
with open('./vectors.pickle', 'wb') as vec_file:
    pickle.dump(vectors, vec_file)

In [29]:
r'''
Compare scores and cosine similarity between expert and model feature vector
'''

for i, row in marks.iterrows():
    fname = row['Case'].split('.')[0]    
    print(f'File {row["Case"]}')
    print(f'Sample 1: {row["Sample 1"]} / {(vectors[fname][0] * vectors[fname][1]).sum().item()}')
    print(f'Sample 2: {row["Sample 2"]} / {(vectors[fname][0] * vectors[fname][2]).sum().item()}')
    print(f'Sample 3: {row["Sample 3"]} / {(vectors[fname][0] * vectors[fname][3]).sum().item()}')
    print('---------------------------------------')

File 00000072_000.png
Sample 1: 1 / -0.06276200711727142
Sample 2: 5 / 0.9999997615814209
Sample 3: 1 / -0.07965496182441711
---------------------------------------
File 00000150_002.png
Sample 1: 5 / 0.9996593594551086
Sample 2: 5 / 0.9980742931365967
Sample 3: 3 / 0.9852204918861389
---------------------------------------
File 00000181_061.png
Sample 1: 4 / 0.9256361722946167
Sample 2: 4 / 0.9171059131622314
Sample 3: 3 / 0.3865862190723419
---------------------------------------
File 00000211_019.png
Sample 1: 4 / 0.9019522666931152
Sample 2: 4 / 0.9366953372955322
Sample 3: 2 / 0.4156748354434967
---------------------------------------
File 00000211_041.png
Sample 1: 3 / 0.4643249213695526
Sample 2: 5 / 0.8481517434120178
Sample 3: 2 / 0.4721382260322571
---------------------------------------
File 00000344_003.png
Sample 1: 2 / -0.3441147208213806
Sample 2: 3 / 0.22616490721702576
Sample 3: 1 / 0.9749670624732971
---------------------------------------
File 00000468_033.png
Sample

In [32]:
test_dataset = 
with torch.no_grad():
    for i, row in pd.read_csv('../Dataset/SecretPart_dummy.csv').iterrows():
        fname = row['Case'].split('.')[0]

        pred_exp = train_dataset[rev_idx[fname]]
        pred_1 = train_dataset[rev_idx[fname] + len(train_dataset.filenames)]
        pred_2 = train_dataset[rev_idx[fname] + 2 * len(train_dataset.filenames)]
        pred_3 = train_dataset[rev_idx[fname] + 3 * len(train_dataset.filenames)]
        
        feature_exp = model.encode(torch.tensor(pred_exp).unsqueeze(0)).numpy()
        feature_1 = model.encode(torch.tensor(pred_1).unsqueeze(0)).numpy()
        feature_2 = model.encode(torch.tensor(pred_2).unsqueeze(0)).numpy()
        feature_3 = model.encode(torch.tensor(pred_3).unsqueeze(0)).numpy()
        
        vectors[fname] = [feature_exp, feature_1, feature_2, feature_3]

KeyError: '00011827_003'